In [2]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique - 1) for _ in range(length)]

In [4]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [5]:
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define padded target sequence
        target = source[:n_out]
        target.reverse() # just a rule for generating target signals
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        '''
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
        ''' 
        src_encoded = to_categorical([source], num_classes=cardinality)[0]
        tar_encoded = to_categorical([target], num_classes=cardinality)[0]
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)[0]

        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)

In [6]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0, 0, :])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [7]:
# configure problem
n_features = 50 + 1 # cardinality
n_steps_in = 6
n_steps_out = 3
# define model
# train, infenc, infdec = define_models(n_features, n_features, 128)

In [8]:
from keras.models import load_model
infenc = load_model('ses_model_infenc.h5')
infdec = load_model('ses_model_infdec.h5')
#infenc.load('ses_model_infenc.h5')

/home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [10]:
infenc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 51)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 128), (None, 128) 92160     
Total params: 92,160
Trainable params: 92,160
Non-trainable params: 0
_________________________________________________________________


In [9]:
infdec.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 51)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  92160       input_2[0][0]                    
                                                                 input_3[0][0]                    
          

In [11]:
# spot check some examples
for _ in range(10):
    X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
    print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

X=[6, 20, 20, 12, 33, 1] y=[20, 20, 6], yhat=[20, 20, 6]
X=[45, 37, 3, 35, 36, 1] y=[3, 37, 45], yhat=[3, 37, 45]
X=[45, 7, 7, 43, 28, 25] y=[7, 7, 45], yhat=[7, 7, 45]
X=[50, 17, 7, 6, 24, 22] y=[7, 17, 50], yhat=[7, 17, 50]
X=[43, 7, 33, 35, 36, 3] y=[33, 7, 43], yhat=[33, 7, 43]
X=[37, 24, 13, 14, 47, 39] y=[13, 24, 37], yhat=[13, 24, 37]
X=[49, 38, 45, 45, 8, 38] y=[45, 38, 49], yhat=[45, 38, 49]
X=[49, 15, 25, 22, 10, 39] y=[25, 15, 49], yhat=[25, 15, 49]
X=[20, 14, 45, 37, 4, 34] y=[45, 14, 20], yhat=[45, 14, 20]
X=[27, 27, 25, 49, 46, 34] y=[25, 27, 27], yhat=[25, 27, 27]
